<a href="https://colab.research.google.com/github/Taniya2711/Data-Analysis-project/blob/main/KDSH_CurrentCoders_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Task1

In [1]:
!pip install textstat # Install the 'textstat' library

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.1/105.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.0 MB/s eta 0:00:00


In [2]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=85eb1a16c55039e83677fde23060906ff5fd227596f20bf794315b0cf92a9caf
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00


In [4]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 102.0 MB/s eta 0:00:00


In [5]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from google.colab import drive
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import pos_tag
from textstat.textstat import textstatistics
import pdfplumber
import optuna


In [17]:
# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Define paths for the dataset
data_path = '/content/drive/My Drive/Research_Papers/Final_folder/References'

Mounted at /content/drive


In [7]:
import os
import pandas as pd
from pdfplumber import open

def extract_text_from_pdf(pdf_path):
  """Extract text from a PDF file."""
  try:
    with open(pdf_path) as pdf:
      text = ''.join(page.extract_text() for page in pdf.pages if page.extract_text())
      return text
  except Exception as e:
    print(f"Error reading {pdf_path}: {e}")
    return ""

def load_data_from_pdfs(base_path):
  """Loads data from PDFs in a structured directory."""
  data = []

  for root, dirs, files in os.walk(base_path):
    for file in files:
      if file.endswith(".pdf"):
        file_path = os.path.join(root, file)
        text = extract_text_from_pdf(file_path)

        # Determine label and conference based on directory structure
        if "Non-publishable" in root:
          conference = "na"  # Extract conference name from path
          label = 0
        else:
          conference = root.split(os.sep)[-1]
          label = 1

        data.append({'text': text, 'label': label, 'conference': conference})

  return pd.DataFrame(data)

# Example usage
base_path = data_path  # Replace with the actual path
data = load_data_from_pdfs(base_path)
print(data)

                                                 text  label conference
0   Examining the Convergence of Denoising Diffusi...      1       TMLR
1   Addressing Min-Max Challenges in Nonconvex-Non...      1       TMLR
2   Published in Transactions on Machine Learning ...      1       TMLR
3   Published in Transactions on Machine Learning ...      1       TMLR
4   Published in Transactions on Machine Learning ...      1       TMLR
5   Detailed Action Identification in Baseball Gam...      1       CVPR
6   Advancements in 3D Food Modeling: A Review of ...      1       CVPR
7   ThisCVPRpaperistheOpenAccessversion,providedby...      1       CVPR
8   FaceTalk: Audio-Driven Motion Diffusion for Ne...      1       CVPR
9   ThisCVPRpaperistheOpenAccessversion,providedby...      1       CVPR
10  The Importance of Written Explanations in\nAgg...      1      EMNLP
11  Advanced techniques for through and contextual...      1      EMNLP
12  Prompts have evil twins\nRimonMelamed LucasH.M...      1    

In [8]:
# Step 3: Preprocess text data
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = text.replace('\n', ' ')  # Remove newlines
    return text

data['text'] = data['text'].apply(preprocess_text)

In [9]:
import nltk
nltk.download('punkt_tab')
# Step 4: Extract Structural, Semantic, and Linguistic Features
def extract_features(df):
    features = {}

    # Structural Features
    features['abstract_length'] = df['text'].apply(lambda x: len(x.split()))
    features['num_sections'] = df['text'].apply(lambda x: x.count('section'))
    features['has_figures'] = df['text'].apply(lambda x: int('figure' in x.lower() or 'fig.' in x.lower()))

    # Linguistic Features
    features['readability'] = df['text'].apply(lambda x: textstatistics().flesch_reading_ease(x))
    features['sentence_complexity'] = df['text'].apply(lambda x: np.mean([len(word_tokenize(sent)) for sent in sent_tokenize(x)]))

    # TF-IDF Features
    tfidf = TfidfVectorizer(max_features=1000)
    tfidf_features = tfidf.fit_transform(df['text']).toarray()

    # Combine all features
    features_df = pd.DataFrame(features)
    combined_features = np.hstack([features_df.values, tfidf_features])

    return combined_features

X_features = extract_features(data)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [10]:
import pandas as pd

# Convert the NumPy array to a Pandas DataFrame
X_features_df = pd.DataFrame(X_features)

# Save the DataFrame to a CSV file
X_features_df.to_csv('X_features.csv', index=False)  # index=False to avoid saving row indices

In [11]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
# Step 5: Tokenize and generate embeddings using SciBERT
tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
from transformers import AutoModel
scibert_model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased')



# ... (Rest of your code, including loading data and defining tokenizer/model) ...

# Tokenization and encoding with batching
def encode_texts_batched(texts, max_length=512, batch_size=32):
    all_encodings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size].tolist()
        encodings = tokenizer(batch_texts,
                              max_length=max_length,
                              truncation=True,
                              padding='max_length',
                              return_tensors='pt')
        all_encodings.append(encodings)
    return all_encodings

encodings = encode_texts_batched(data['text'])

# Generate SciBERT embeddings with batching
def generate_embeddings_batched(encodings_batches):
    all_embeddings = []
    for encodings in encodings_batches:
        with torch.no_grad():
            outputs = scibert_model(**encodings)[0]
            cls_embeddings = outputs[:, 0, :]
            all_embeddings.append(cls_embeddings)
    return torch.cat(all_embeddings, dim=0)

scibert_embeddings = generate_embeddings_batched(encodings)
# Combine embeddings with engineered features
X_combined = np.hstack([X_features, scibert_embeddings.detach().numpy()])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [12]:
import pandas as pd

# Convert the NumPy array to a Pandas DataFrame
X_combined_df = pd.DataFrame(X_combined)

# Save the DataFrame to a CSV file
X_combined_df.to_csv('X_combined.csv', index=False)  # index=False to avoid saving row indices

In [13]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Extract actual conference labels
conference_labels = data['conference'].values

# Step 2: Encode conference labels using LabelEncoder
label_encoder = LabelEncoder()
conference_encoded = label_encoder.fit_transform(conference_labels)

# Step 3: Add the encoded conference labels to X_combined
X_combined_with_conference = np.hstack([X_combined, conference_encoded.reshape(-1, 1)])

# Step 4: Optional - Check the shape of the new dataset and mappings
print("Shape of X_combined with conference labels:", X_combined_with_conference.shape)
print("Conference Label Mappings:", dict(zip(label_encoder.classes_, range(len(label_encoder.classes_)))))


Shape of X_combined with conference labels: (50, 1774)
Conference Label Mappings: {'CVPR': 0, 'EMNLP': 1, 'KDD': 2, 'NeurIPS': 3, 'TMLR': 4, 'na': 5}


In [14]:
import pandas as pd

# Convert the NumPy array to a Pandas DataFrame
X_combined_with_conference_df = pd.DataFrame(X_combined_with_conference)

# Save the DataFrame to a CSV file
X_combined_with_conference_df.to_csv('X_combined_with_conference.csv', index=False)  # index=False to avoid saving row indices


In [15]:
import pandas as pd
import numpy as np

# Load the combined dataset
X_combined_with_conference = pd.read_csv('X_combined_with_conference.csv')

# Load the original data to get the labels
data = pd.read_csv('X_combined_with_conference.csv') # Assuming 'data' DataFrame is saved as 'data.csv'


# Assuming the last column is the conference label
# Separate features and labels
X = X_combined_with_conference.iloc[:, :-1]
y = X_combined_with_conference.iloc[:, 2] # Use the 'label' column from the original data


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
y_test.shape

(40, 1773)
(10, 1773)
(40,)


(10,)

In [16]:
def objective(trial):
    input_shape = X_combined.shape[1]

    # Suggest hyperparameters
    num_units_1 = trial.suggest_int('num_units_1', 64, 256)
    num_units_2 = trial.suggest_int('num_units_2', 32, 128)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)

    # Build model
    inputs = Input(shape=(input_shape,))
    x = Dense(num_units_1, activation='relu')(inputs)
    x = Dropout(dropout_rate)(x)
    x = Dense(num_units_2, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)

    # Compile model
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train model
    model.fit(X_train, y_train, validation_data=(X_test, y_test),
              epochs=50, batch_size=16, callbacks=[early_stopping], verbose=0)

    # Evaluate model
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy
# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

# Best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)


[I 2025-01-13 16:14:58,771] A new study created in memory with name: no-name-bad91140-9d06-420c-a805-9b64628479cd
[I 2025-01-13 16:15:01,181] Trial 0 finished with value: 0.800000011920929 and parameters: {'num_units_1': 137, 'num_units_2': 36, 'dropout_rate': 0.46838352172462083, 'learning_rate': 0.0008903882946444563}. Best is trial 0 with value: 0.800000011920929.
[I 2025-01-13 16:15:03,060] Trial 1 finished with value: 0.800000011920929 and parameters: {'num_units_1': 114, 'num_units_2': 89, 'dropout_rate': 0.3150853972394836, 'learning_rate': 9.134766429610108e-05}. Best is trial 0 with value: 0.800000011920929.
[I 2025-01-13 16:15:04,886] Trial 2 finished with value: 0.800000011920929 and parameters: {'num_units_1': 253, 'num_units_2': 58, 'dropout_rate': 0.17025913344164806, 'learning_rate': 0.0006080547024181054}. Best is trial 0 with value: 0.800000011920929.
[I 2025-01-13 16:15:06,141] Trial 3 finished with value: 0.800000011920929 and parameters: {'num_units_1': 151, 'num_un

Best Hyperparameters: {'num_units_1': 193, 'num_units_2': 44, 'dropout_rate': 0.20289847943202888, 'learning_rate': 0.000689627033329429}


In [17]:
# Define the model using the best hyperparameters
def build_model(input_shape, params):
    inputs = Input(shape=(input_shape,))
    x = Dense(params['num_units_1'], activation='relu')(inputs)
    x = Dropout(params['dropout_rate'])(x)
    x = Dense(params['num_units_2'], activation='relu')(x)
    x = Dropout(params['dropout_rate'])(x)
    outputs = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Build and train the final model
final_model = build_model(X_combined.shape[1], best_params)
final_model.fit(X_train, y_train, validation_data=(X_test, y_test),
                epochs=50, batch_size=16, verbose=1)


Epoch 1/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.7453 - loss: 13.3961 - val_accuracy: 0.8000 - val_loss: 6.3607
Epoch 2/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6453 - loss: 10.7229 - val_accuracy: 0.8000 - val_loss: 3.3260
Epoch 3/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.5219 - loss: 15.9344 - val_accuracy: 0.8000 - val_loss: 5.5422
Epoch 4/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.6828 - loss: 8.6043 - val_accuracy: 0.8000 - val_loss: 6.7667
Epoch 5/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7547 - loss: 5.7343 - val_accuracy: 0.8000 - val_loss: 6.6202
Epoch 6/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7422 - loss: 9.1500 - val_accuracy: 0.8000 - val_loss: 5.2692
Epoch 7/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - accuracy: 0.6734 - loss: 11.3535 - val_accuracy: 0.8000 - val_loss: 4.0739
Epoch 8/50
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.8219 - loss: 3.6361 - val_accuracy: 0.8000 - val_loss: 2.5

In [18]:
from sklearn.metrics import accuracy_score
predictions= final_model.predict(X_test)
predictions = (predictions > 0.5).astype(int)
accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Accuracy: 0.9


In [19]:
from sklearn.metrics import f1_score

# ... (Your existing code) ...

# Evaluate the final model
predictions = final_model.predict(X_test)
predictions = (predictions > 0.5).astype(int)

# Calculate the F1-score
f1 = f1_score(y_test, predictions)
print(f"F1 Score: {f1}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
F1 Score: 0.9411764705882353


In [20]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


# Mount Google Drive
# drive.mount('/content/drive')

# Define the path to the 'papers' folder
papers_folder = "/content/drive/My Drive/Research_Papers/Final_folder/Papers"  # Replace with your actual path


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [21]:
import os
import pandas as pd
from pdfplumber import open

def extract_text_from_pdf(pdf_path):
  """Extract text from a PDF file."""
  try:
    with open(pdf_path) as pdf:
      text = ''.join(page.extract_text() for page in pdf.pages if page.extract_text())
      return text
  except Exception as e:
    print(f"Error reading {pdf_path}: {e}")
    return ""

def load_data_from_pdfs(base_path):
  """Loads data from PDFs in a structured directory."""
  data = []

  for root, dirs, files in os.walk(base_path):
    for file in files:
      if file.endswith(".pdf"):
        file_path = os.path.join(root, file)
        text = extract_text_from_pdf(file_path)

        # Determine label and conference based on directory structure
        # if "Non-publishable" in root:
        #   conference = "na"  # Extract conference name from path
        #   label = 0
        # else:
        #   conference = root.split(os.sep)[-1]
        #   label = 1

        data.append({'Paper ID': file,'text':text})

  return pd.DataFrame(data)

# Example usage
val_path = papers_folder  # Replace with the actual path
val_data = load_data_from_pdfs(val_path)
print(val_data)

     Paper ID                                               text
0    P033.pdf  AMR Parsing using Stack-LSTMs\nAbstract\nWepre...
1    P126.pdf  Designing Data Markets Using Deep Learning\nTe...
2    P129.pdf  Xray Emissions and their Consequential Effects...
3    P003.pdf  Explainable Reinforcement Learning for Financi...
4    P100.pdf  Engine Performance and its Implications for\nM...
..        ...                                                ...
130  P006.pdf  High-Throughput Genomic Sequencing in Marine\n...
131  P034.pdf  Enhanced Normalization in Vision Transformers:...
132  P106.pdf  Next-Generation Brain-Computer Interfaces for\...
133  P088.pdf  Analyzing Groups of Neurons in Neural Networks...
134  P035.pdf  Game-Theoretic Optimization for Crowdsourced\n...

[135 rows x 2 columns]


In [22]:
# Step 3: Preprocess text data
def preprocess_text(text):
    text = text.lower()  # Lowercase
    text = text.replace('\n', ' ')  # Remove newlines
    return text

val_data['text'] = val_data['text'].apply(preprocess_text)

In [23]:
import nltk
nltk.download('punkt_tab')
# Step 4: Extract Structural, Semantic, and Linguistic Features
def extract_features(df):
    features = {}

    # Structural Features
    features['abstract_length'] = df['text'].apply(lambda x: len(x.split()))
    features['num_sections'] = df['text'].apply(lambda x: x.count('section'))
    features['has_figures'] = df['text'].apply(lambda x: int('figure' in x.lower() or 'fig.' in x.lower()))

    # Linguistic Features
    features['readability'] = df['text'].apply(lambda x: textstatistics().flesch_reading_ease(x))
    features['sentence_complexity'] = df['text'].apply(lambda x: np.mean([len(word_tokenize(sent)) for sent in sent_tokenize(x)]))

    # TF-IDF Features
    tfidf = TfidfVectorizer(max_features=1000)
    tfidf_features = tfidf.fit_transform(df['text']).toarray()

    # Combine all features
    features_df = pd.DataFrame(features)
    combined_features = np.hstack([features_df.values, tfidf_features])

    return combined_features

X_val_features = extract_features(val_data)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [24]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel

# ... (Rest of your code, including loading data and defining tokenizer/model) ...

# Tokenization and encoding with batching
def encode_texts_batched(texts, max_length=512, batch_size=32):
    all_encodings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i+batch_size].tolist()
        encodings = tokenizer(batch_texts,
                              max_length=max_length,
                              truncation=True,
                              padding='max_length',
                              return_tensors='pt')
        all_encodings.append(encodings)
    return all_encodings

val_encodings_batched = encode_texts_batched(val_data['text'])

# Generate SciBERT embeddings with batching
def generate_embeddings_batched(encodings_batches):
    all_embeddings = []
    for encodings in encodings_batches:
        with torch.no_grad():
            outputs = scibert_model(**encodings)[0]
            cls_embeddings = outputs[:, 0, :]
            all_embeddings.append(cls_embeddings)
    return torch.cat(all_embeddings, dim=0)

val_scibert_embeddings = generate_embeddings_batched(val_encodings_batched)
# Combine embeddings with engineered features
val_X_combined = np.hstack([X_val_features, val_scibert_embeddings.detach().numpy()])

# ... (Rest of your code, combining embeddings with features) ...

In [25]:
# Make predictions on the validation data
val_predictions = final_model.predict(val_X_combined)
val_predictions = (val_predictions > 0.5).astype(int)

#Rename the column
val_data = val_data.rename(columns={'Paper ID': 'PDF Name'})

#Create the 'Publishable' column in val_data and populate it with predictions
val_data['Publishable'] = val_predictions

# Remove the 'text' column
# val_data = val_data.drop('text', axis=1)

# Process 'PDF Name' column
val_data['PDF Name'] = val_data['PDF Name'].str.replace(r'.pdf$', '', regex=True)
# val_data['PDF Name'] = val_data['PDF Name'].str.zfill(3)

#Save the updated DataFrame to a new CSV file
val_data.to_csv('val_data_with_predictions.csv', index=False)

print(val_data.head())

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step  
  PDF Name                                               text  Publishable
0     P033  amr parsing using stack-lstms abstract weprese...            0
1     P126  designing data markets using deep learning tec...            1
2     P129  xray emissions and their consequential effects...            1
3     P003  explainable reinforcement learning for financi...            0
4     P100  engine performance and its implications for ma...            1


In [26]:
import pandas as pd

# Load the CSV file
val_data = pd.read_csv('val_data_with_predictions.csv')

# Process 'PDF Name' column to keep only one 'P'
val_data['PDF Name'] = val_data['PDF Name'].str.replace(r'(P)\1+', r'\1', regex=True)

# Sort the DataFrame by 'PDF Name' in ascending order
val_data = val_data.sort_values(by='PDF Name', ascending=True)

# Reset the index
val_data = val_data.reset_index(drop=True)

# Save the updated DataFrame to a new CSV file
val_data.to_csv('val_data_with_predictions_modified.csv', index=False)

print(val_data.head())

  PDF Name                                               text  Publishable
0     P001  leveraging clustering techniques for enhanced ...            0
1     P002  virus propagation and their far-reaching impli...            1
2     P003  explainable reinforcement learning for financi...            0
3     P004  graph neural networks without training: harnes...            0
4     P005  collaborative clothing segmentation and identi...            1


In [27]:
val_data= val_data.rename(columns={'PDF Name': 'Paper ID'})

# Save the updated DataFrame to a new CSV file
val_data.to_csv('results.csv', index=False)

print(val_data.head())

  Paper ID                                               text  Publishable
0     P001  leveraging clustering techniques for enhanced ...            0
1     P002  virus propagation and their far-reaching impli...            1
2     P003  explainable reinforcement learning for financi...            0
3     P004  graph neural networks without training: harnes...            0
4     P005  collaborative clothing segmentation and identi...            1


#Task2

In [1]:
!pip install huggingface-hub

In [2]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.0 MB/s eta 0:00:00


In [6]:
import os
import os
import pandas as pd
from PyPDF2 import PdfReader
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from transformers import pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [18]:
# Constants for file paths
REFERENCE_FOLDER = "/content/drive/My Drive/Research_Papers/Final_folder/References/Publishable"
RESULTS_CSV_PATH = "/content/drive/My Drive/Research_Papers/Final_folder/results(1).csv"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
LLM_MODEL = "gpt-2"

In [19]:
# Load results.csv file
results_df = pd.read_csv(RESULTS_CSV_PATH)

# Filter publishable papers
publishable_papers = results_df[results_df['Publishable'] == 1]

# Path to reference papers
reference_path = REFERENCE_FOLDER

# Define conferences and their themes
conference_themes = {
    'CVPR': "computer vision, image processing, object detection, deep learning in vision",
    'KDD': "data mining, big data analytics, knowledge discovery, machine learning",
    'EMNLP': "natural language processing, computational linguistics, language models",
    'NeurIPS': "neural networks, reinforcement learning, AI advancements, computational neuroscience",
    'TMLR': "machine learning research, theoretical analysis, model evaluation, reproducibility",
}

In [20]:
# Extract text from reference papers and publishable papers
def extract_text_from_papers(reference_path, publishable_papers):
    all_texts = []  # Store all texts for creating a common vocabulary
    reference_texts = {}
    for conference in os.listdir(reference_path):
        conference_folder = os.path.join(reference_path, conference)
        if os.path.isdir(conference_folder):
            reference_texts[conference] = []
            for file in os.listdir(conference_folder):
                file_path = os.path.join(conference_folder, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        text = f.read()
                        reference_texts[conference].append(text)
                        all_texts.append(text) # Add to all_texts
                except UnicodeDecodeError:
                    with open(file_path, 'r', encoding='latin-1') as f:
                        text = f.read()
                        reference_texts[conference].append(text)
                        all_texts.append(text) # Add to all_texts

    # Add publishable paper texts to all_texts
    for _, row in publishable_papers.iterrows():
        all_texts.append(row['text'])

    return reference_texts, all_texts

reference_texts, all_texts = extract_text_from_papers(reference_path, publishable_papers)

# Create TF-IDF vectors using a common vocabulary
vectorizer = TfidfVectorizer(stop_words='english')
vectorizer.fit(all_texts)  # Fit on all texts to create a common vocabulary

conference_vectors = {
    conference: vectorizer.transform(texts)
    for conference, texts in reference_texts.items()
}

In [21]:
# Load a Hugging Face LLM pipeline for rationale generation
rationale_pipeline = pipeline("text-generation", model="gpt2")

# Generate rationale for each publishable paper
rationales = []
for _, row in publishable_papers.iterrows():
    paper_title = row['text'].split('abstract')[0]  # Extract title before the abstract
    paper_content = row['text']

    # Compute similarity with each conference using the common vocabulary
    paper_vector = vectorizer.transform([paper_content])
    similarities = {
        conference: cosine_similarity(paper_vector, vectors).mean()
        for conference, vectors in conference_vectors.items()
    }

    # Select conference with highest similarity
    selected_conference = max(similarities, key=similarities.get)

    # Generate rationale for the selected conference
    rationale_prompt = (
        f"The paper titled '{paper_title}' has been classified as publishable. "
        f"It has been recommended for the conference '{selected_conference.upper()}'. "
        f"The paper's content discusses {conference_themes[selected_conference]}. "
        f"Provide a well-reasoned rationale (up to 100 words) explaining how the paper's "
        f"content, methodology, and findings align with the themes, focus areas, and quality "
        f"standards of '{selected_conference.upper()}'."
    )

    rationale = rationale_pipeline(rationale_prompt, max_new_tokens=200, num_return_sequences=1)[0]['generated_text']
    # Remove the prompt from the generated output
    rationale_cleaned = rationale.replace(rationale_prompt, "").strip()

    # Truncate to the first 100 words
    rationale_truncated = " ".join(rationale_cleaned.split()[:100])
    rationales.append({'Paper ID': row['Paper ID'],'Publishable': row['Publishable'], 'Conference': selected_conference, 'rationale': rationale_truncated})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:5

In [22]:
# Save results with rationales to a CSV file
output_df = pd.DataFrame(rationales)
output_df.to_csv('classified_papers_with_rationales.csv', index=False)

print("Classification and rationale generation complete. Results saved to 'classified_papers_with_rationales.csv'.")

Classification and rationale generation complete. Results saved to 'classified_papers_with_rationales.csv'.


In [23]:
# Load the updated dataframe
results = pd.read_csv("classified_papers_with_rationales.csv")

# Group by conference and count the number of rows
conference_counts = results.groupby('Conference').size().reset_index(name='Row_Count')

# Print the results
conference_counts

,Conference,Row_Count
0,CVPR,15
1,EMNLP,6
2,KDD,8
3,NeurIPS,1
4,TMLR,9


In [24]:
# Create new columns for conference and rationale
results = pd.read_csv(RESULTS_CSV_PATH)
results['Conference'] = 'NA'
results['Rationale'] = 'NA'

# Update columns based on publishable status
# Use enumerate to track the index within 'recommendations'
for rec_index, (_, row) in enumerate(results[results['Publishable'] == 1].iterrows()):
    # Only iterate over publishable papers
    paper_id = row['Paper ID']
    paper_text = row['text']
    # Access recommendations using rec_index
    results.loc[results['Paper ID'] == paper_id, 'Conference'] = rationales[rec_index]['Conference']
    results.loc[results['Paper ID'] == paper_id, 'Rationale'] = rationales[rec_index]['rationale']

# Save the updated dataframe to the same CSV file
results.drop(columns='text',inplace=True)
results.to_csv("results.csv", index=False)